In [196]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import GRU, Dropout, Masking, Embedding, Dense, Lambda, BatchNormalization
import xml.etree.ElementTree as ET
import pandas as pd
import importlib
importlib.reload(constants)

class Predictor:
    def __init__(self):
        pass

    def getModel(self):
        model = keras.Sequential()
#         model.add(keras.layers.Masking())
        model.add(Dense(constants.hidden_dim, activation='tanh'))
        model.add(BatchNormalization())
#         model.add(Dropout(0.2))
        model.add(
            GRU(constants.round_num, return_sequences=True))
        model.add(
            GRU(constants.round_num))
        model.add(Dense(constants.hidden_dim, activation='tanh'))
        model.add(Dense(4, activation='tanh'))
        
#         inp = keras.Input(shape=(constants.timestep, 4), batch_size=constants.batch_size)
        inp = keras.Input(shape=(constants.round_num, constants.pred_emb_dim))
        output = model(inp)
        return keras.Model(inputs=inp, outputs = output)



predictor = Predictor()
model = predictor.getModel()
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mse', metrics=['mse'])
model.summary()


# inputs = tf.random.normal((constants.batch_size, constants.round_num, 11))
# inputs.shape == model.inputs.shape
# print(model(inputs))

Model: "functional_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 15, 15)]          0         
_________________________________________________________________
sequential_22 (Sequential)   (None, 4)                 19209     
Total params: 19,209
Trainable params: 18,809
Non-trainable params: 400
_________________________________________________________________


In [33]:
import pandas as pd
from logs_parser.parser import parse_mjlog
from logs_parser.viewer import print_node
import xml.etree.ElementTree as ET
import preprocess
import sys
import json
import numpy as np
import os

directory = "/Users/xingyu/Documents/USC/courses/ml game527/trainingdata"
pdlist = []
for filename in os.listdir(directory):

    if filename.endswith("csv"):
        print(os.path.join(directory, filename))
        pdlist.append(pd.read_csv(os.path.join(directory, filename)))
df = pd.concat(pdlist)
print(len(df))

/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2021.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2009.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2019.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2018.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2015.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2014.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2016.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2017.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2013.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2012.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2010.csv
/Users/xingyu/Documents/USC/courses/ml game527/trainingdata/2011.csv
567997


In [184]:
features = []
targets = []
from tqdm import tqdm


for i, xml_str in tqdm(enumerate(df['log_content'])):
    try:
        scores = []
        node = ET.fromstring(xml_str)
        data = parse_mjlog(node)
        if len(data["meta"]['UN'][3]["name"]) == 0:
            continue
        dans = np.array([player['dan'] for player in data['meta']['UN']])
        uma = data['rounds'][-1][-1]['data']['result']['uma']


        feature = []
        for i in range(len(data["rounds"])):

            round_data = data["rounds"][i]
            stats = round_data[-1]['data']
            round_info = preprocess.get_round_info(round_data)

            round_score = np.array(stats['scores'])/1e5
            gains = np.array(round_info['gains'])/1e5
            dealer = int(round_info['dealer'])
            repeat_dealer = stats['ba']['combo']
            riichi_bets = stats['ba']['reach']
            embed = np.concatenate((round_score, gains, dans, np.array([dealer, repeat_dealer, riichi_bets])), axis=0)

            feature.append(embed)
        feature = [np.zeros(constants.pred_emb_dim)] * max(constants.round_num-len(feature), 0) + feature
        features.append(np.array(feature[-constants.round_num:]))
        targets.append(uma)        
    except:
        continue
targets = np.array(targets) / 200.0


567997it [52:23, 180.72it/s]


In [86]:
from math import floor
tmp = list(zip(features, targets))
random.shuffle(tmp)
size = len(tmp)
# train_valid = floor(0.8 * size)
# train = tmp[:train_valid]
# valid = tmp[train_valid:]

In [148]:
from tensorflow.keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator
from sklearn.preprocessing import normalize
import random
# import importlib
# importlib.reload(constants)



# def Generator(data, batch_size):
#     while True:
#         random.shuffle(data)
#         features, targets, cnt = [], [], 0
#         for rounds in data:
#             history, final_score = rounds
#             for i in range(1, min(len(history), constants.round_num)):
#                 padded = np.zeros((constants.round_num, constants.pred_emb_dim))
#                 padded[-i:] = np.array(history[:i])
#                 features.append(padded)
#                 targets.append(final_score)
#                 cnt = (cnt+1)%batch_size
#                 if cnt == 0:
#                     yield np.array(features), np.array(targets)
#                     features, targets = [], []
                    
# g = Generator(list(zip(features, targets)), constants.batch_size)


model.fit(np.asarray(features), np.asarray(targets), epochs=3, batch_size=constants.batch_size, validation_split=0.2)
# model.fit_generator(g, epochs=2, steps_per_epoch=4 * len(targets) // constants.batch_size)
# model.save('pred_model')
#0.0079
# window_size = constants.timestep
# for x, y in Generator(res):
#     print(x, y)
#     break
# for X, y in batchGenerator(res, constants.batch_size):
#     history = model.fit(X, y, epochs=3)
#     break




ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [164]:
features[0]

array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.25, 0.24, 0.25, 0.25, 0.25, 0.24, 0.25, 0.25, 0, 0, 1,
        list([16, 16, 16, 16])],
       [0.25, 0.37, 0.12, 0.25, 0.25, 0.37, 0.12, 0.25, 1, 0, 1,
        list([16, 16, 16, 16])],
       [0.23, 0.33, 0.21, 0.22, 0.23, 0.33, 0

In [197]:
model.fit(np.array(features), np.array(targets), epochs=1, batch_size=constants.batch_size)

8202/8202 [==============================] - 112s 14ms/step - loss: 0.0018 - mse: 0.0018


In [199]:
model.save('pred_model')

INFO:tensorflow:Assets written to: pred_model/assets


In [198]:
model.predict(np.asarray(features[:3]))

array([[-0.04452683,  0.19589661, -0.12874329, -0.0315387 ],
       [-0.17844862,  0.02504468, -0.10141908,  0.24641648],
       [ 0.02343977, -0.06120011, -0.18054199,  0.20731613]],
      dtype=float32)

In [191]:
targets[:3]

array([[-0.08 ,  0.205, -0.15 ,  0.025],
       [-0.185,  0.04 , -0.115,  0.26 ],
       [ 0.045, -0.065, -0.205,  0.225]])